# Reconocimiento de dígitos con KNN

Veamos como interactuar con las clases implementadas en el módulo de C++ haciendo una experimentación muy simplificada, y a modo de ejemplo, del método KNN para reconocer digitos

In [1]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import metnum
import numpy

/home/sebasbocaccio/Desktop/FACU/Metodos/tp2-metodos/tp2/notebooks
Python 3.6.5 :: Anaconda, Inc.


# Importar datos

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from sklearn.metrics import accuracy_score
df_train = pd.read_csv("../data/train.csv")

Es un csv, con columnas por cada pixel

### Reducir el dataset

Ya que el dataset es grande, me quedo con una porción de éste

**<font color=red>ESTO ES SOLO PARA HACER PRUEBAS. CUANDO REPORTEN LOS RESULTADOS, USAR DATASET COMPLETO</font>**

In [3]:
df_train = df_train[:4000]

En el primer caracter está el dígito a reconocer. Llamamos a esto `y_train`

In [3]:
# Uso values para mandar todo a arrays de numpy
X = df_train[df_train.columns[1:]].values
y = df_train["label"].values.reshape(-1, 1)

X.shape, y.shape

((42000, 784), (42000, 1))

Convierto el resto a matrices de 28 x 28

A ver cómo se ven...

### Balance de clases

¿Tenemos la misma cantidad de cada número?

In [4]:
df_train["label"].value_counts(normalize=True)

1    0.111524
7    0.104786
3    0.103595
9    0.099714
2    0.099452
6    0.098500
0    0.098381
4    0.096952
8    0.096738
5    0.090357
Name: label, dtype: float64

No: tenemos más 1 que 5, pero tampoco está tan desbalanceado

### Conjunto de development

In [5]:
limit = int(0.8 * X.shape[0]) 

X_train, y_train = X[:limit], y[:limit]

dist = []
for i in range(0,len(X_train),1):
    dist.append((numpy.inner(X_train[i], X_train[i]),y_train[i][0]))
dist.sort()
for i in dist:
    print(i[1])
X_val, y_val = X[limit:], y[limit:]

assert len(X_train) == len(y_train)
assert len(X_val) == len(y_val)

print(f"Ahora tengo {len(X_train)} instancias de entrenamiento y {len(X_val)} de validación")


1
1
1
6
1
1
1
1
1
1
1
1
5
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
1
1
1
1
1
1
1
1
9
1
1
1
2
1
1
1
1
1
1
1
1
3
1
1
1
1
1
1
1
1
1
1
7
3
1
1
1
1
1
1
1
1
1
1
1
9
1
1
1
1
9
1
1
1
1
1
1
1
1
1
1
1
1
1
5
1
1
1
1
1
1
1
9
1
1
1
1
1
1
1
4
1
1
4
1
1
6
1
1
1
1
1
1
1
1
1
4
1
1
1
1
1
1
3
3
1
1
1
1
4
1
1
7
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
7
1
1
6
7
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
1
1
1
4
1
1
1
1
1
1
1
6
1
1
1
1
8
1
5
1
1
1
1
1
5
1
1
1
1
1
1
5
1
7
7
1
1
5
1
1
1
1
1
1
1
9
1
5
1
1
1
7
1
1
1
1
1
1
1
5
1
1
1
1
4
1
7
1
7
1
1
1
7
1
7
5
1
7
1
1
1
1
1
9
1
9
1
7
1
3
1
3
1
1
1
3
1
1
1
1
1
6
1
1
1
3
7
1
1
1
1
1
1
1
1
1
1
7
5
1
5
1
1
7
1
1
1
1
9
7
1
5
7
7
1
1
4
5
1
1
1
1
1
1
1
1
7
2
1
1
1
1
1
1
1
1
1
9
1
1
5
1
1
1
1
1
1
1
1
1
5
1
1
1
7
1
1
1
1
3
4
1
1
1
1
9
1
1
1
2
1
1
1
1
1
4
1
1
1
5
1
1
1
1
1
7
1
1
1
1
7
1
4
1
1
1
1
1
1
1
1
4
4
1
5
1
1
2
1
1
6
1
1
1
1
5
1
1
4
1
0
1
1
1
1
1
1
5
1
6
1
1
1
7
1
1
1
3
1
2
1
9
1
1
1
1
1
1
1
1
1
1
7
1
1
1
1
1
1
3
7
1
1
1
1
1
1


1
1
1
7
7
5
7
3
5
1
4
6
2
5
1
4
9
2
3
6
1
6
3
1
9
4
8
9
6
5
1
4
3
8
0
1
1
7
1
9
4
9
1
4
4
1
4
1
9
1
8
4
8
4
7
5
4
1
4
1
6
9
0
9
9
1
7
7
3
9
8
1
5
9
5
3
7
4
5
4
3
1
8
2
1
5
6
5
1
1
1
8
6
1
4
9
2
1
8
5
9
1
5
5
3
5
1
7
7
1
4
4
4
1
2
1
1
9
7
4
1
6
7
4
1
1
1
1
8
5
8
9
1
3
3
1
7
4
1
1
1
4
4
7
3
4
7
1
6
9
1
7
7
3
5
1
1
5
5
4
5
2
2
7
1
7
3
1
3
4
1
7
7
2
9
9
7
5
4
3
3
3
1
7
5
9
2
9
3
7
9
7
4
1
3
9
7
5
1
1
7
6
6
2
9
1
1
2
1
5
7
6
1
1
1
3
1
7
7
7
7
1
7
1
5
5
7
9
1
5
4
1
2
1
1
2
1
2
1
9
4
1
1
9
9
1
1
7
1
8
3
1
7
4
9
6
8
1
7
4
4
1
9
1
0
5
7
9
4
3
9
2
1
5
2
9
9
1
7
5
5
4
7
4
2
7
2
4
1
1
8
4
3
3
4
9
6
6
4
3
2
9
1
1
1
9
1
8
9
6
2
4
1
7
7
7
2
7
7
1
6
1
7
8
9
7
9
1
9
4
7
1
6
8
4
4
3
1
7
9
4
6
1
1
5
6
6
4
7
7
8
4
5
4
1
7
1
2
5
5
1
1
7
1
4
2
1
1
6
5
6
8
4
7
0
1
7
4
8
1
1
1
7
7
2
4
3
1
9
1
5
5
7
5
1
4
1
4
9
5
9
1
1
8
1
6
1
1
8
2
1
1
1
3
3
9
1
3
9
7
7
1
1
5
1
2
9
4
7
3
1
1
7
1
1
1
4
5
7
4
1
1
3
4
6
1
1
8
7
1
7
6
7
1
7
2
7
8
9
5
1
4
8
9
1
7
1
7
5
6
4
3
9
6
7
2
8
1
7
2
5
4
8
1
7
5
2
1
5
8
4
9
4
1
2
3
3
7
3
8


4
3
1
7
7
1
7
2
5
7
3
4
8
7
7
2
6
4
3
0
5
7
7
4
3
7
6
5
5
4
4
5
4
7
7
2
9
7
0
9
7
4
6
2
8
4
2
1
2
1
7
7
4
9
1
9
5
6
7
5
1
7
9
2
7
4
3
8
8
7
4
0
1
4
5
9
7
5
2
7
7
2
7
7
8
6
7
4
5
4
9
4
2
1
7
2
1
8
8
9
1
1
4
0
2
9
7
4
3
9
7
3
9
3
5
7
6
8
7
6
8
5
9
7
4
7
0
9
8
2
7
8
9
7
3
7
7
0
1
1
9
5
9
5
7
6
5
7
4
8
8
4
3
6
9
1
6
1
8
4
7
4
4
4
7
8
6
3
7
6
4
8
6
7
5
9
2
6
0
5
9
6
5
5
9
7
2
8
1
7
3
7
6
4
9
1
7
7
2
4
6
8
1
7
7
5
4
6
7
0
3
7
4
1
4
9
0
9
9
4
9
5
4
1
0
5
1
7
1
7
2
3
2
6
8
4
2
3
1
5
3
2
2
7
9
9
4
1
6
7
8
3
5
4
8
3
7
9
8
6
7
6
2
6
1
2
5
1
5
7
8
5
6
7
9
7
4
7
6
1
9
5
7
0
3
4
7
1
6
5
7
6
1
7
7
4
2
7
4
6
4
3
7
2
9
7
9
1
7
4
3
2
9
1
3
9
6
5
1
1
8
4
8
3
9
1
4
0
5
2
9
8
1
4
5
2
1
4
6
4
6
1
6
1
5
1
4
2
7
7
8
0
1
3
6
5
6
4
4
7
4
9
6
5
4
7
7
4
7
9
7
3
7
9
5
6
5
3
8
4
2
8
7
9
1
3
4
5
6
2
3
4
4
5
1
4
4
9
4
7
5
1
7
6
6
4
9
7
6
0
1
7
4
7
9
9
7
4
3
8
6
3
7
4
7
4
6
9
7
9
7
9
7
6
7
7
7
8
5
9
2
9
2
7
7
7
9
1
5
7
7
5
9
8
6
7
5
7
7
9
7
9
1
3
6
9
4
9
1
9
8
4
6
5
3
3
1
5
4
0
5
3
6
7
8
5
6
4
5
6
0
8
7
2
4
5
6
3
6
3


2
4
9
9
8
4
4
3
4
8
4
8
5
4
4
3
2
5
4
6
3
8
5
2
0
3
7
4
0
0
4
1
6
8
9
7
7
8
9
7
8
2
3
7
3
6
6
9
5
6
8
8
7
7
4
9
9
0
6
3
9
3
0
6
7
0
6
7
3
9
5
0
3
3
9
5
6
3
5
9
8
1
6
5
9
6
3
6
4
7
2
4
7
3
3
5
8
6
3
2
6
5
4
5
4
1
2
5
5
3
5
9
6
2
0
0
3
3
7
2
6
7
0
2
3
6
1
0
2
1
9
4
7
4
4
4
8
3
8
2
3
8
0
4
6
8
6
4
4
4
6
9
0
2
6
4
7
6
5
3
7
1
2
9
2
9
8
8
0
8
4
5
9
6
9
8
4
7
7
4
7
8
3
0
7
9
7
9
3
3
4
9
9
9
4
5
7
3
6
5
5
0
8
5
0
3
9
7
3
8
7
9
1
9
6
2
3
5
6
5
7
9
2
3
0
0
8
6
4
9
6
9
6
9
9
5
5
4
9
1
6
9
6
2
6
9
9
6
7
9
3
2
0
5
7
2
9
5
9
5
7
0
7
9
1
2
7
5
8
3
4
4
5
7
0
6
9
8
3
3
9
3
2
4
6
1
2
3
4
4
0
4
8
3
5
3
7
9
9
7
6
6
3
7
7
6
7
0
0
8
1
4
2
9
1
5
7
6
2
9
4
3
7
8
3
8
1
5
4
7
7
8
9
5
5
5
5
3
9
3
6
6
7
7
9
9
6
0
6
5
3
0
5
2
5
6
2
6
7
9
8
6
9
2
5
3
3
3
9
8
9
5
3
8
5
1
1
2
8
6
3
4
7
5
3
9
8
2
4
5
9
2
7
7
2
3
7
0
7
2
8
8
8
5
4
9
1
4
5
5
2
7
8
7
0
0
9
2
6
4
2
2
2
9
7
7
9
8
4
4
7
9
2
4
8
2
7
6
7
4
2
7
9
2
7
2
3
9
9
3
4
9
8
5
6
2
9
5
3
8
2
8
4
8
5
7
2
1
3
4
9
6
8
5
9
4
8
4
9
5
7
6
7
5
5
0
3
7
0
9
2
0
5
2
8
9
4
2
7
4


8
8
4
6
8
3
1
4
7
3
1
4
8
8
8
8
0
4
2
2
2
4
2
4
6
8
2
4
3
0
6
9
8
4
2
0
7
4
7
2
3
0
8
4
3
3
6
7
0
5
3
6
1
2
3
0
0
0
7
0
9
7
0
2
3
4
5
1
7
2
6
3
7
0
5
5
9
4
2
7
7
4
9
3
8
7
3
4
7
5
5
3
6
0
4
8
9
5
9
2
3
3
0
5
3
5
9
3
2
2
9
8
5
3
4
4
6
7
8
6
9
7
2
4
2
0
6
0
7
2
9
0
7
3
5
2
3
9
5
7
8
8
3
7
5
0
3
4
4
6
3
7
6
5
1
7
6
9
8
3
0
4
2
2
8
8
3
2
6
5
3
7
4
0
6
3
8
5
2
3
8
2
6
6
5
6
1
5
4
3
9
0
0
2
9
3
1
9
3
0
7
7
0
4
0
6
0
7
6
4
3
4
2
5
3
0
2
9
6
6
4
5
0
7
2
8
5
3
5
2
4
2
8
7
8
9
0
9
6
9
0
9
0
8
3
0
4
5
2
0
3
2
3
9
0
3
0
3
3
7
6
7
3
8
9
8
6
9
7
3
0
4
6
2
4
4
3
0
5
6
2
3
7
2
2
8
9
5
4
5
9
0
1
2
9
9
5
6
9
2
9
3
7
7
7
5
7
2
5
6
8
3
6
6
7
7
2
8
5
0
8
7
0
4
4
2
5
5
7
7
7
2
4
7
2
5
8
8
4
7
0
7
6
7
8
0
5
1
8
6
0
9
4
2
2
6
4
6
2
5
9
9
7
2
6
5
6
5
9
9
6
7
4
7
5
7
5
8
0
3
2
6
4
5
8
5
9
3
5
6
7
6
2
4
7
9
2
4
6
2
8
8
9
3
0
9
8
9
1
0
5
8
6
5
9
3
3
7
9
8
3
8
5
5
2
9
7
6
7
7
5
4
2
2
2
5
4
0
9
5
6
5
0
0
3
5
6
6
3
9
8
7
6
2
0
6
8
3
3
3
9
3
4
6
4
8
2
5
7
4
2
5
2
5
2
2
8
9
8
5
6
3
8
8
4
9
8
9
3
5
3
7
7
8
6
5
6
7
2
5


5
3
5
8
8
2
0
5
4
0
2
5
5
2
3
9
9
9
2
3
8
8
9
6
2
6
0
8
1
3
9
5
9
4
3
5
0
8
2
6
9
2
7
2
3
8
8
9
0
8
8
2
6
4
3
7
0
5
0
5
4
0
5
6
5
8
2
9
0
6
9
3
3
8
2
3
3
8
3
3
2
4
4
9
9
7
2
5
8
5
4
2
4
0
6
8
6
2
6
5
3
8
7
2
7
5
1
9
7
2
3
0
5
6
3
8
8
3
0
7
8
8
7
3
0
5
2
2
2
8
8
0
0
8
4
3
4
8
8
7
9
5
5
8
8
9
4
7
4
6
6
2
3
2
7
4
7
9
9
5
8
5
9
2
9
9
5
5
9
8
8
6
7
5
0
0
4
2
9
2
0
2
3
0
0
0
3
3
6
7
4
0
5
2
5
2
9
6
0
6
2
0
3
6
8
8
7
4
0
3
7
7
4
7
6
0
0
3
5
5
9
8
3
2
4
0
8
8
2
0
4
2
6
0
5
2
7
6
4
4
6
9
2
6
3
3
9
6
3
2
2
8
8
7
5
8
4
0
8
6
5
8
8
6
9
9
5
8
3
4
7
0
4
6
8
6
5
9
4
2
0
7
7
2
9
9
3
3
4
7
7
2
4
8
4
3
5
8
4
5
3
3
8
5
3
0
8
6
4
8
0
5
3
9
9
2
4
6
4
8
4
4
7
5
3
9
0
2
8
0
4
9
7
8
5
3
0
4
8
2
2
2
6
8
4
9
8
9
3
2
4
2
3
0
6
4
2
8
3
8
4
3
8
4
7
9
6
7
6
6
9
3
8
0
8
3
8
8
0
3
7
8
6
0
4
7
0
9
6
2
2
4
0
2
2
9
6
3
3
0
2
8
8
7
3
6
0
2
2
3
6
9
5
3
7
8
6
5
3
9
0
8
9
3
6
8
5
3
4
3
7
8
7
0
2
8
4
8
9
8
3
2
6
3
3
7
0
2
8
9
3
3
2
8
0
6
0
0
3
2
0
3
4
4
4
2
2
3
4
9
6
0
0
9
9
3
3
8
8
3
6
0
8
3
7
9
4
7
3
8
9
3
8
6
2
6
7
2
3
0


2
7
2
9
6
8
0
5
2
6
9
8
0
8
2
4
0
8
8
2
5
6
3
2
2
0
0
3
6
4
2
0
0
6
3
8
0
2
0
0
3
6
5
3
8
0
6
8
8
8
8
8
8
0
2
5
0
3
0
6
8
4
6
0
3
7
8
3
5
5
8
3
6
6
3
2
0
6
5
9
8
5
0
8
8
6
8
0
8
6
5
0
9
0
6
0
3
2
5
3
0
2
4
0
8
8
0
6
3
3
8
3
0
8
8
0
9
0
3
0
5
3
8
5
0
0
5
0
0
5
0
8
2
9
9
0
0
2
6
9
8
0
5
0
8
0
5
3
4
9
2
2
3
2
8
8
2
9
0
2
2
3
2
8
2
2
2
0
2
6
9
2
6
5
3
0
4
0
0
5
0
0
8
2
2
8
4
0
0
4
6
2
4
3
2
3
3
8
3
8
7
8
8
8
2
7
3
8
6
5
7
6
0
6
8
8
0
2
5
8
3
0
2
8
5
0
8
6
4
0
6
0
3
2
2
6
5
3
5
3
6
8
8
0
8
3
0
3
8
9
6
0
5
7
6
6
4
0
6
8
9
2
8
5
6
6
2
0
7
5
4
2
2
0
4
0
0
2
8
0
2
3
0
3
8
8
0
3
6
9
3
0
2
0
3
8
4
0
7
3
0
6
2
5
8
2
0
2
0
6
0
3
5
2
2
0
4
0
8
0
6
8
5
9
0
3
2
7
4
6
3
4
2
4
2
6
0
5
3
0
2
2
0
7
0
6
2
1
4
0
8
2
8
0
3
5
0
2
8
3
8
3
2
2
6
3
3
0
5
2
6
2
7
4
0
6
8
0
0
2
3
0
2
9
4
8
0
3
0
8
2
0
0
0
3
4
2
8
8
3
0
2
4
5
0
6
6
2
2
0
6
0
0
4
5
0
8
0
5
9
9
0
9
2
2
7
4
2
0
3
0
5
6
2
5
9
6
4
1
4
3
0
0
0
2
2
8
5
2
3
4
0
3
2
3
3
2
6
2
2
3
0
8
0
0
3
6
4
5
6
0
3
3
9
4
0
5
0
7
9
3
5
2
2
6
0
2
8
4
8
6
6
0
2
9
2
2
5
3
0


In [7]:
from sklearn.neighbors import KNeighborsClassifier
import time
# Acá pueden cambiar el clasificador nuestro por el de sklearn!

clf = metnum.KNNClassifier(1)
#clf = KNeighborsClassifier(1)
setup_time_start_time = time.time()
clf.fit(X_train, y_train)
setup_time_end_time = time.time()
setup_time = setup_time_end_time-setup_time_start_time

In [8]:
%%time
from sklearn.metrics import accuracy_score
predict_time_start_time = time.time()
y_pred = clf.predict(X_val)
predict_time_end_time = time.time()
predict_time = predict_time_end_time-predict_time_start_time
print(predict_time + setup_time)
dist = []
for i in range(0,len(X_val),1):
    dist.append((numpy.inner(X_val[i], X_val[i])))
acc = accuracy_score(y_val, y_pred)
print("Accuracy: {}".format(acc))


360.35698413848877
Accuracy: 0.9673809523809523
CPU times: user 5min 48s, sys: 646 ms, total: 5min 49s
Wall time: 5min 48s


# Testeo

In [11]:
import pandas as pd
df_train = pd.read_csv("../data/custom_test.csv")



# Uso values para mandar todo a arrays de numpy
X = df_train[df_train.columns[1:]].values
y = df_train["label"].values.reshape(-1, 1)

X.shape, y.shape

x_train = X[:3]
y_train = y[:3]
x_test = X[3:]
y_test = y[3:]

clf = metnum.KNNClassifier(2)
clf.fit(x_train, y_train)


In [12]:

y_pred = clf.predict(x_test)
y_pred

array([1., 1., 1.])

# Testeo 2 

In [1]:
def function():
    return 4